# Generate data samples

:::{warning}

The {doc}`pycompwa </index>` is no longer maintained. Use the [ComPWA](https://compwa-org.rtfd.io) packages [QRules](https://qrules.rtfd.io), [AmpForm](https://ampform.rtfd.io), and [TensorWaves](https://tensorwaves.rtfd.io) instead!

:::

There are two different types of data samples: (1) {ref}`phase space samples <usage/tools/generate:Phase space samples>` and (2) {ref}`intensity-based samples <usage/tools/generate:Intensity-based samples>` (which mimic the intensity distributions of the reaction). In addition, you may want to generate more data points in specific regions of phase space (e.g. in the case of a sharp resonance). This is called {ref}`importance sampling <usage/tools/generate:Importance sampling>`.

Usually, you need a phase space sample to generate an intensity-based sample, so we start with that.

## Phase space samples

With the {mod}`pycompwa.ui` module you can easily create various types of Monte Carlo phase space samples. You do this with the function {func}`.generate_phsp`.

One of the required arguments is a {class}`.PhaseSpaceEventGenerator` instance. There are currently two options: {class}`.EvtGenGenerator` and {class}`.RootGenerator`. Using the {class}`.EvtGenGenerator` is recommended for numerical precision:

```{eval-rst}
.. autoclass:: pycompwa.ui.EvtGenGenerator
    :special-members: __init__
    :noindex:
```

As you can see, such a {class}`.PhaseSpaceEventGenerator` requires state transition info from a {class}`.Kinematics` instance, such as the {class}`.HelicityKinematics` class. 

The second argument for the {func}`.generate_phsp` function is a random number generator. Here, there are again two options: {class}`.StdUniformRealGenerator` and {class}`.RootUniformRealGenerator`.

To illustrate this procedure, we first need to {doc}`create some kinematics model </usage/workflow/step1>`:

In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.ERROR)  # not interested in warnings now

In [ ]:
from pycompwa.expertsystem.amplitude.helicitydecay import (
    HelicityAmplitudeGeneratorXML,
)
from pycompwa.expertsystem.ui.system_control import (
    InteractionTypes,
    StateTransitionManager,
)

initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma"), ("pi0"), ("pi0")]
tbd_manager = StateTransitionManager(
    initial_state,
    final_state,
    formalism_type="helicity",
    topology_building="isobar",
)

tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong, InteractionTypes.EM]
)
tbd_manager.allowed_intermediate_particles = ["f"]

graph_interaction_settings_groups = tbd_manager.prepare_graphs()
solutions, _ = tbd_manager.find_solutions(graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")

xml_generator = HelicityAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file("model.xml")

Now we can go through the procedure explained above:

In [ ]:
import pycompwa.ui as pwa

In [ ]:
# specify kinematics
kinematics = pwa.create_helicity_kinematics("model.xml")

# specify generators
generator = pwa.EvtGenGenerator(
    kinematics.get_particle_state_transition_kinematics_info()
)
random_generator = pwa.StdUniformRealGenerator(12345)

# generate phase space sample
phsp_sample = pwa.generate_phsp(1000, generator, random_generator)

## Intensity-based samples

Data samples are more complicated than phase space samples in that they represent the intensity profile resulting from a reaction. You therefore need an {class}`.Intensity` object and a phase space over which to generate that intensity distribution. We therefore call such a data sample an **intensity-based sample**.

An intensity-based sample is generated with the function {func}`~.ui.generate`. Its usage is similar to {func}`.generate_phsp`, but now you have to give an {class}`.Intensity` and a {class}`.Kinematics` instance as additional arguments. Sampling is performed through hit & miss, which is reflected by the efficiency printed at the end.

In [ ]:
# create intensity profile
particle_list = pwa.read_particles("model.xml")
intensity_builder = pwa.IntensityBuilderXML(
    "model.xml", particle_list, kinematics, phsp_sample
)
intensity = intensity_builder.create_intensity()

# generate data
data_sample = pwa.generate(
    100, kinematics, generator, intensity, random_generator
)

## Importance sampling

In the example above, we used the decay $J/\psi \to \gamma \pi^0 \pi^0$ with f resonances only. Imagine, however, that we want to investigate the resonance $\omega \to \gamma\pi^0$. This resonance has such a narrow peak that you will run into problems when normalizing the intensity model.

To normalize a distribution, you need know the integral over that distribution. The integral over an intensity distribution distribution can only be computed numerically. We simply do this by computing the sum of the intensities evaluated for each point in a phase space (Monte Carlo integration), and averaging over the number of phase space points. This means that the higher the number of phase space points, the more accurate your integral.

With a narrow peak, however, many of the points in an evenly distributed phase space sample will lie outside the peak region, where the intensity (and therefore their contribution to the integral) is highest. So in order to have a good estimate for the integral over this narrow peak structure, you want to have a larger number of phase space points under that peak. Now, you could just use a bigger phase space sample (increasing number of points within the peak region), but this will be resource intensive as it will result in many hits in regions that hardly contributes to the integral.

A smarter strategy is generate a sample that has a higher density of phase space points within the peak region. This strategy is called **importance sampling**.

We first again create an amplitude model, this time including a 'widened' $\omega$ resonance (see also {doc}`particles`). Widening $\omega$ is only done so that the resonance is still visible―the peak is still extremely narrow.

In [ ]:
from pycompwa.expertsystem.amplitude.helicitydecay import (
    HelicityAmplitudeGeneratorXML,
)
from pycompwa.expertsystem.state.particle import particle_list
from pycompwa.expertsystem.ui.system_control import (
    InteractionTypes,
    StateTransitionManager,
)

initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma"), ("pi0"), ("pi0")]
tbd_manager = StateTransitionManager(
    initial_state,
    final_state,
    formalism_type="helicity",
    topology_building="isobar",
)

# Widen omega resonance
omega = particle_list["omega(782)"]
parameters = omega["DecayInfo"]["Parameter"]
for par in parameters:
    if par["@Type"] == "Width":
        par["Value"] = 0.001

tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong, InteractionTypes.EM]
)
tbd_manager.allowed_intermediate_particles = ["f2(1270)", "omega"]
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
solutions, _ = tbd_manager.find_solutions(graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")

xml_generator = HelicityAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file("model.xml")

Now we generate a phase space sample that is importance sampled by the
intensity. This can be done with the
{func}`.generate_importance_sampled_phsp` function.

:::{note}

Note that you still need a 'normal' phase space sample for the {class}`.IntensityBuilderXML`. In theory, however, this shouldn't be necessary at this stage, but in the current implementation, this sample is required for computing the outermost intensity. Luckily, during data generation the parameters of the model stay fixed, so this normalization is irrelevant and just causes a constant scaling factor.

:::

In [ ]:
import pycompwa.ui as pwa

In [ ]:
# specify kinematics
particle_list = pwa.read_particles("model.xml")
kinematics = pwa.create_helicity_kinematics("model.xml", particle_list)

# specify generators
generator = pwa.EvtGenGenerator(
    kinematics.get_particle_state_transition_kinematics_info()
)
random_generator = pwa.StdUniformRealGenerator(12345)

# generate evenly distributed phase space sample
phsp_sample = pwa.generate_phsp(100000, generator, random_generator)

# create intensity
intensity_builder = pwa.IntensityBuilderXML(
    "model.xml", particle_list, kinematics, phsp_sample
)
intensity = intensity_builder.create_intensity()

# generate importance-sampled phase space sample
phsp_sample_importance = pwa.generate_importance_sampled_phsp(
    2000, kinematics, generator, intensity, random_generator
)

:::{warning}

The current implementation is simple and uniformly generates events in the phase space domain. Depending on the shape of your Intensity, this can be quite computation intensive. However, ideally you would only run this once for a specific reaction.

:::

The effect of this importance sampling can be best seen in a Dalitz plot. There, one can see that there are more events in the regions where the intensity is large (the resonances), but one still sees a uniform distribution overall.

To create a Dalitz plot, we first have to create a {class}`.DataSet`, as we saw in {ref}`this section <usage/workflow/step4:4.2 Visualize>`:

In [ ]:
kinematics.create_all_subsystems()
phsp_set = kinematics.convert(phsp_sample)
phsp_set_importance = kinematics.convert(phsp_sample_importance)

We can then use {mod}`matplotlib.pyplot` to create the Dalitz plot. Note that we use {func}`.replace_ids` to transform the final state IDs into readable particles names.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from pycompwa.data import naming

In [ ]:
def dalitz_plot(data_set, mass_x, mass_y, bins=50, **kwargs):
    """Helper function to create a Dalitz plot with useful axis titles."""
    plt.hist2d(
        data_set.data[mass_x],
        data_set.data[mass_y],
        weights=data_set.weights,
        bins=bins,
        **kwargs,
    )
    plt.xlabel(naming.replace_ids(mass_x, kinematics))
    plt.ylabel(naming.replace_ids(mass_y, kinematics))

In [ ]:
dalitz_plot(
    phsp_set_importance, "mSq_(3,4)", "mSq_(2,4)", bins=80, norm=LogNorm()
)
plt.gca().set_title("importance-sampled phase space");

Note how this compares to the evenly distributed phase space sample and the intensity-based sample:

In [ ]:
dalitz_plot(phsp_set, "mSq_(3,4)", "mSq_(2,4)", bins=80, norm=LogNorm())
plt.gca().set_title("evenly distributed phase space");

In [ ]:
pwa.Logging("error")
data_sample = pwa.generate(
    2000, kinematics, generator, intensity, random_generator
)
data_set = kinematics.convert(data_sample)
dalitz_plot(data_set, "mSq_(3,4)", "mSq_(2,4)", bins=80, norm=LogNorm())
plt.gca().set_title("intensity-based sample");